In [ ]:
import cekComputerLabs as cek
cek.checkGitRepo()

# Bomb Calorimetry Virtual Experiment

## Procedure
In this virtual lab instead you will have to
1. Choose your sample (Benzoic acid)
2. Create a tablet for you sample 
3. "Perform" the experiment (obtain temperature inside the bomb calorimeter vs time)
4. Fit the temperature profile before and after the experiment to obtain the temperature change
5. Determine the calorimeter heat capacity of the bomb calorimeter to calibrate the instrument
6. Repeat 1 to 5 to obtain an average value for the heat capacity of the bomb calorimeter
7. Repeat 1 to 4 for sucrose and naphtalene
8. Determine the combustion enthalpy and formation enthalpy of Sucrose and Napthalene

#### Assume the wire described in the video releases a negligible amount of heat

In [ ]:
cek.bombCalorimetryLab()

# Working notebooks

In [ ]:
cek.launchNotebooks()